In [1]:
import pandas as pd
import numpy as np

In [2]:
path = r"C:\Users\yanzh\Desktop\code_and_data\4. Deep learning part\predict probability for each scenario.csv"
scenarios = pd.read_csv(path)
scenarios

,demand,probability
0,3285,0.539734
1,3042,0.399869
2,2379,0.031845
3,3116,0.021401
4,3002,0.007152


In [3]:
location_ids = [48, 68, 79, 142, 161, 163, 170, 186, 234, 236, 237, 239, 74, 231, 88, 41, 50, 12, 113, 243, 244, 232, 43, 194, 140]

def df_filtering_yellow(df):

    selected_dates = ['2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01']
    selected_dates = [pd.to_datetime(date).date() for date in selected_dates]

    df = df[df['tpep_pickup_datetime'].dt.date.isin(selected_dates)]
    df = df[df['tpep_pickup_datetime'].dt.hour == 21]
    
    # Filter the Dataframe
    df = df[df['PULocationID'].isin(location_ids) & df['DOLocationID'].isin(location_ids)]

    # Drop the data from useless columns
    df = df.drop(columns=['VendorID', 'RatecodeID', 'store_and_fwd_flag', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge'])

    # Delete the data that travel distance = 0, beacasue we don't consider the situation where trips in the same zone. This data still need to be reconsidered.
    df = df[df['PULocationID'] != df['DOLocationID']]
    
    # Delete the data that fare amount is negative.
    df = df[df['fare_amount'] >= 0]

    # Delete the data that total_amount is negative
    df = df[df['total_amount'] > 0]

    # Delete the data that trip distance larger than 200
    df = df[df['trip_distance'] <= 200]

    # Additional condition: Filter out rows where dropoff time exceeds pickup time by more than 5 hours
    df = df[(df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']) <= pd.Timedelta(hours=5)]

    return df


In [4]:
import glob

# Define the path and pattern to match the yellow_tripdata files
file_pattern = "C:/Users/yanzh/Desktop/code_and_data/4. Deep learning part/原始数据/January 1st 1h/yellow_tripdata_*.parquet"

# Use glob to get all file paths matching the pattern
file_paths = glob.glob(file_pattern)

# Read each file and store the DataFrames in a list
dataframes = [df_filtering_yellow(pd.read_parquet(file)) for file in file_paths]

# Concatenate all DataFrames into a single DataFrame
df_yellow = pd.concat(dataframes, ignore_index=True)

In [5]:
df_yellow = df_yellow[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID']]
df_yellow = df_yellow.rename(columns={'tpep_pickup_datetime': 'starting_time',
                                      'tpep_dropoff_datetime': 'ending_time',
                                      'PULocationID': 'starting_point',
                                      'DOLocationID': 'ending_point'})
df_yellow

,starting_time,ending_time,starting_point,ending_point
0,2015-01-01 21:02:35,2015-01-01 21:08:13,142,237
1,2015-01-01 21:29:19,2015-01-01 21:37:49,236,142
2,2015-01-01 21:56:24,2015-01-01 22:01:06,234,161
3,2015-01-01 21:15:25,2015-01-01 21:21:12,113,79
4,2015-01-01 21:59:18,2015-01-01 22:06:49,161,163
...,...,...,...,...
9384,2019-01-01 21:59:18,2019-01-01 22:07:58,68,48
9385,2019-01-01 21:35:22,2019-01-01 21:43:05,161,142
9386,2019-01-01 21:57:36,2019-01-01 22:10:15,48,113
9387,2019-01-01 21:59:49,2019-01-01 22:07:41,237,239


In [6]:
def df_filtering_green(df):

    selected_dates = ['2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01']
    selected_dates = [pd.to_datetime(date).date() for date in selected_dates]

    df = df[df['lpep_pickup_datetime'].dt.date.isin(selected_dates)]
    df = df[df['lpep_pickup_datetime'].dt.hour == 21]
    
    # Filter the Dataframe
    df = df[df['PULocationID'].isin(location_ids) & df['DOLocationID'].isin(location_ids)]

    # Drop the data from useless columns
    df = df.drop(columns=['VendorID', 'RatecodeID', 'store_and_fwd_flag', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge'])

    # Delete the data that travel distance = 0, beacasue we don't consider the situation where trips in the same zone. This data still need to be reconsidered.
    df = df[df['PULocationID'] != df['DOLocationID']]
    
    # Delete the data that fare amount is negative.
    df = df[df['fare_amount'] >= 0]

    # Delete the data that total_amount is negative
    df = df[df['total_amount'] > 0]

    # Delete the data that trip distance larger than 200
    df = df[df['trip_distance'] <= 200]

    # Additional condition: Filter out rows where dropoff time exceeds pickup time by more than 5 hours
    df = df[(df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']) <= pd.Timedelta(hours=5)]

    return df

In [7]:
# Define the path and pattern to match the yellow_tripdata files
file_pattern = "C:/Users/yanzh/Desktop/code_and_data/4. Deep learning part/原始数据/January 1st 1h/green_tripdata_*.parquet"

# Use glob to get all file paths matching the pattern
file_paths = glob.glob(file_pattern)

# Read each file and store the DataFrames in a list
dataframes = [df_filtering_green(pd.read_parquet(file)) for file in file_paths]

# Concatenate all DataFrames into a single DataFrame
df_green = pd.concat(dataframes, ignore_index=True)

In [8]:
df_green = df_green[['lpep_pickup_datetime', 'lpep_dropoff_datetime', 'PULocationID', 'DOLocationID']]
df_green = df_green.rename(columns={'lpep_pickup_datetime': 'starting_time',
                                      'lpep_dropoff_datetime': 'ending_time',
                                      'PULocationID': 'starting_point',
                                      'DOLocationID': 'ending_point'})
df_green

,starting_time,ending_time,starting_point,ending_point
0,2015-01-01 21:34:37,2015-01-01 21:38:22,74,43
1,2015-01-01 21:48:51,2015-01-01 21:55:28,41,74
2,2015-01-01 21:55:59,2015-01-01 22:02:08,74,41
3,2015-01-01 21:50:13,2015-01-01 22:09:49,41,170
4,2015-01-01 21:29:36,2015-01-01 21:40:27,74,244
...,...,...,...,...
245,2019-01-01 21:35:04,2019-01-01 21:47:09,74,237
246,2019-01-01 21:25:01,2019-01-01 21:42:19,41,48
247,2019-01-01 21:10:47,2019-01-01 21:27:10,74,142
248,2019-01-01 21:22:28,2019-01-01 21:31:04,41,74


In [9]:
df_combined = pd.concat([df_yellow, df_green], ignore_index=True)
df_combined

,starting_time,ending_time,starting_point,ending_point
0,2015-01-01 21:02:35,2015-01-01 21:08:13,142,237
1,2015-01-01 21:29:19,2015-01-01 21:37:49,236,142
2,2015-01-01 21:56:24,2015-01-01 22:01:06,234,161
3,2015-01-01 21:15:25,2015-01-01 21:21:12,113,79
4,2015-01-01 21:59:18,2015-01-01 22:06:49,161,163
...,...,...,...,...
9634,2019-01-01 21:35:04,2019-01-01 21:47:09,74,237
9635,2019-01-01 21:25:01,2019-01-01 21:42:19,41,48
9636,2019-01-01 21:10:47,2019-01-01 21:27:10,74,142
9637,2019-01-01 21:22:28,2019-01-01 21:31:04,41,74


In [10]:
scenarios

,demand,probability
0,3285,0.539734
1,3042,0.399869
2,2379,0.031845
3,3116,0.021401
4,3002,0.007152


here we don't consider assign total demand in each specific scenario, just sample number of total demand from df_combined

In [11]:
for idx, row in scenarios.iterrows():
    sample_size = int(row['demand'])
    sampled_trips = df_combined.sample(n=sample_size, random_state=42)
    sampled_trips.to_csv(f'scenairo {idx}.csv', index=False)